# Experimento 5 

Hiperparmetro: num_val_windows = 2

In [28]:
import pandas as pd
import numpy as np
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [29]:
df = pd.read_csv("../entregable/datasets/periodo_x_producto_con_target_transformado_con_feature_engineering.csv", sep=',', encoding='utf-8')
df

,product_id,periodo,nacimiento_producto,muerte_producto,mes_n,total_meses,producto_nuevo,ciclo_de_vida_inicial,sku_size,stock_final,...,elecciones_legislativas_1,elecciones_legislativas_2,crisis_cambiaria_1,crisis_cambiaria_2,las_paso_2019_1,las_paso_2019_2,devaluacion_post_PASO,elecciones_presidenciales_2019,ipc,dolar
0,20001,201701,201701,201912,1,36,0,0,3000.0,NaN,...,0,0,0,0,0,0,0,0,"1,6","16,24"
1,20001,201702,201701,201912,2,36,0,0,3000.0,NaN,...,0,0,0,0,0,0,0,0,"2,1","15,88"
2,20001,201703,201701,201912,3,36,0,0,3000.0,NaN,...,0,0,0,0,0,0,0,0,"2,3","15,58"
3,20001,201704,201701,201912,4,36,0,0,3000.0,NaN,...,0,0,0,0,0,0,0,0,"2,7","15,59"
4,20001,201705,201701,201912,5,36,0,0,3000.0,NaN,...,0,0,0,0,0,0,0,0,"1,4","15,99"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31357,21281,201704,201702,201708,3,7,1,1,NaN,NaN,...,0,0,0,0,0,0,0,0,"2,7","15,59"
31358,21281,201705,201702,201708,4,7,1,0,NaN,NaN,...,0,0,0,0,0,0,0,0,"1,4","15,99"
31359,21281,201706,201702,201708,5,7,1,0,NaN,NaN,...,0,0,0,0,0,0,0,0,"1,2","16,44"
31360,21281,201707,201702,201708,6,7,1,0,NaN,NaN,...,0,0,0,0,0,0,0,0,"1,8","17,65"


In [30]:
dfg = df.groupby(['periodo', 'product_id']).agg({'tn': 'sum'}).reset_index()
dfg.head()

,periodo,product_id,tn
0,201701,20001,934.77222
1,201701,20002,550.15707
2,201701,20003,1063.45835
3,201701,20004,555.91614
4,201701,20005,494.27011


In [31]:
dfg = dfg[dfg['periodo'] < 201911]

In [32]:
dfg['periodo_dt'] = pd.to_datetime(dfg['periodo'].astype(str), format='%Y%m')
dfg.rename(columns={'tn': 'target', 'product_id':'item_id', 'periodo_dt': 'timestamp'}, inplace=True)
dfg.drop(columns=['periodo'], inplace=True)

In [33]:
productos_ok = pd.read_csv('../../data/raw/product_id_apredecir201912.csv', sep=',')
productos_ok.head()

,product_id
0,20001
1,20002
2,20003
3,20004
4,20005


In [34]:
dfg = dfg[dfg['item_id'].isin(productos_ok['product_id'].unique())]

In [35]:
data = TimeSeriesDataFrame(dfg)
data

,,target
item_id,timestamp,
20001,2017-01-01,934.77222
20002,2017-01-01,550.15707
20003,2017-01-01,1063.45835
20004,2017-01-01,555.91614
20005,2017-01-01,494.27011
...,...,...
21263,2019-10-01,0.01552
21265,2019-10-01,0.10921
21266,2019-10-01,0.11831


In [39]:
def entrenar_con_semillerio(data, semillas=[42, 101, 202, 303, 404], prediction_length=2):
    """
    Entrena múltiples modelos con diferentes semillas y devuelve una lista de predictores
    
    Args:
        data: TimeSeriesDataFrame con datos de entrenamiento
        semillas: Lista de semillas a utilizar
        prediction_length: Horizonte de predicción
        
    Returns:
        Lista de predictores entrenados
    """
    predictors = []
    
    for seed in semillas:
        print(f"\nEntrenando con semilla {seed}")
        
        predictor = TimeSeriesPredictor(
            target='target',
            prediction_length=prediction_length,
            freq="M",
        )
        
        predictor.fit(
            data,
            num_val_windows=2,
            presets="medium_quality",
            random_seed=seed,  # Semilla para AutoGluon
            # time_limit=3600  # Límite de tiempo por modelo (ajustar)
        )
        
        predictors.append(predictor)
    
    return predictors

In [40]:
def predecir_con_semillerio(predictors, data, future_covariates=None):
    """
    Genera predicciones promediando los resultados de múltiples predictores
    
    Args:
        predictors: Lista de predictores entrenados
        data: Datos históricos
        future_covariates: Variables exógenas futuras
        
    Returns:
        TimeSeriesDataFrame con predicciones combinadas
    """
    all_predictions = []
    
    for predictor in predictors:
        preds = predictor.predict(data)
        all_predictions.append(preds)
    
    # Combinar predicciones (promedio)
    combined = all_predictions[0].copy()
    for col in combined.columns:
        if col not in ["item_id", "timestamp"]:
            combined[col] = np.mean([p[col] for p in all_predictions], axis=0)
    
    return combined

In [41]:
# Configuración
semillas = [42, 101, 202, 303, 404]
prediction_length = 2

# 1. Entrenamiento con semillerio
predictors = entrenar_con_semillerio(data, semillas, prediction_length)

# 2. Preparar variables exógenas futuras (ajusta esto a tus datos)
# future_covariates = TimeSeriesDataFrame.from_data_frame(
#     pd.DataFrame({
#         'timestamp': pd.date_range(start=data.index.get_level_values('timestamp').max() + pd.DateOffset(months=1),
#         periods=prediction_length,
#         freq="M"),
#         'dolar': [valores_reales_o_proyectados_dolar],
#         'ipc': [valores_reales_o_proyectados_ipc]
#     }),
#     id_column="item_id",
#     timestamp_column="timestamp"
# )

# 3. Predicción ensemble
predictions = predecir_con_semillerio(predictors, data)

# 4. Guardar resultados
predictions.reset_index()[["item_id", "timestamp", "mean"]].to_csv("predicciones_semillerio.csv", index=False)

Frequency 'M' stored as 'ME'
Beginning AutoGluon training...
AutoGluon will save models to 'c:\Users\Usuario\Documents\Universidad\austral\2025\Lab3\Lab3-MCD\notebooks\model_autogluon\AutogluonModels\ag-20250621_194531'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.4
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          4
GPU Count:          0
Memory Avail:       2.62 GB / 15.89 GB (16.5%)
Disk Space Avail:   418.81 GB / 893.49 GB (46.9%)
Setting presets to: medium_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'ME',
 'hyperparameters': 'light',
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 42,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'verbosity': 2}




Entrenando con semilla 42


train_data with frequency 'None' has been resampled to frequency 'ME'.
Provided train_data has 20815 rows, 780 time series. Median time series length is 34 (min=2, max=34). 
	Removing 120 short time series from train_data. Only series with length >= 9 will be used for training.
	After filtering, train_data has 20176 rows, 660 time series. Median time series length is 34 (min=9, max=34). 

Provided data contains following columns:
	target: 'target'

AutoGluon will gauge predictive performance using evaluation metric: 'WQL'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2025-06-21 16:45:37
Models that will be trained: ['Naive', 'SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'ETS', 'Theta', 'Chronos[bolt_small]', 'TemporalFusionTransformer']
Training timeseries model Naive. 
	-0.2651       = Validation score (-WQL)
	38.92   s     = Training runtime
	1.21   

AttributeError: _ARRAY_API not found

	numpy.core.multiarray failed to import
Fitting simple weighted ensemble.
	Ensemble weights: {'Chronos[bolt_small]': np.float64(0.08),
 'DirectTabular': np.float64(0.05),
 'ETS': np.float64(0.34),
 'RecursiveTabular': np.float64(0.12),
 'SeasonalNaive': np.float64(0.19),
 'Theta': np.float64(0.21)}
	-0.1990       = Validation score (-WQL)
	2.67    s     = Training runtime
	22.15   s     = Validation (prediction) runtime
Training complete. Models trained: ['Naive', 'SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'ETS', 'Theta', 'Chronos[bolt_small]', 'WeightedEnsemble']
Total runtime: 186.91 s
Best model: WeightedEnsemble
Best model score: -0.1990
c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\autogluon\timeseries\predictor.py:197: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  std_freq = pd.tseries.frequencies.to_offset(self.freq).freqstr
Frequency 'M' stored as 'ME'
Beginning AutoGluon training...


...

	numpy.core.multiarray failed to import
Fitting simple weighted ensemble.
	Ensemble weights: {'Chronos[bolt_small]': np.float64(0.09),
 'DirectTabular': np.float64(0.04),
 'ETS': np.float64(0.34),
 'RecursiveTabular': np.float64(0.13),
 'SeasonalNaive': np.float64(0.19),
 'Theta': np.float64(0.21)}
	-0.1991       = Validation score (-WQL)
	2.54    s     = Training runtime
	18.17   s     = Validation (prediction) runtime
Training complete. Models trained: ['Naive', 'SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'ETS', 'Theta', 'Chronos[bolt_small]', 'WeightedEnsemble']
Total runtime: 120.28 s
Best model: WeightedEnsemble
Best model score: -0.1991
c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\autogluon\timeseries\predictor.py:197: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  std_freq = pd.tseries.frequencies.to_offset(self.freq).freqstr
Frequency 'M' stored as 'ME'
Beginning AutoGluon training...



Entrenando con semilla 404


Provided train_data has 20815 rows, 780 time series. Median time series length is 34 (min=2, max=34). 
	Removing 120 short time series from train_data. Only series with length >= 9 will be used for training.
	After filtering, train_data has 20176 rows, 660 time series. Median time series length is 34 (min=9, max=34). 

Provided data contains following columns:
	target: 'target'

AutoGluon will gauge predictive performance using evaluation metric: 'WQL'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2025-06-21 16:55:20
Models that will be trained: ['Naive', 'SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'ETS', 'Theta', 'Chronos[bolt_small]', 'TemporalFusionTransformer']
Training timeseries model Naive. 
	-0.2651       = Validation score (-WQL)
	4.91    s     = Training runtime
	0.86    s     = Validation (prediction) runtime
Training timeseries model Seas

AttributeError: _ARRAY_API not found

	numpy.core.multiarray failed to import
Fitting simple weighted ensemble.
	Ensemble weights: {'Chronos[bolt_small]': np.float64(0.09), 'DirectTabular': np.float64(0.05), 'ETS': np.float64(0.34), 'RecursiveTabular': np.float64(0.12), 'SeasonalNaive': np.float64(0.19), 'Theta': np.float64(0.2)}
	-0.1990       = Validation score (-WQL)
	2.43    s     = Training runtime
	20.11   s     = Validation (prediction) runtime
Training complete. Models trained: ['Naive', 'SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'ETS', 'Theta', 'Chronos[bolt_small]', 'WeightedEnsemble']
Total runtime: 123.10 s
Best model: WeightedEnsemble
Best model score: -0.1990
data with frequency 'None' has been resampled to frequency 'ME'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble
data with frequency 'None' has been resampled to frequency 'ME'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble
d

In [42]:
# predictor = TimeSeriesPredictor(target='target', prediction_length=2, freq="M").fit(data, num_val_windows = 2)
# predictions = predictor.predict(data)
predictions

mean          0.1          0.2          0.3  \
item_id timestamp                                                        
20001   2019-11-30  1602.979685  1181.474029  1324.053262  1425.180978   
        2019-12-31  1542.149114  1085.764821  1239.033598  1345.539964   
20002   2019-11-30  1436.318348  1048.714383  1177.464702  1268.195428   
        2019-12-31  1272.765049   862.786457  1002.240448  1099.117145   
20003   2019-11-30  1058.467963   751.287488   850.779777   924.281616   
...                         ...          ...          ...          ...   
20995   2019-12-31     3.319402     0.262788     1.328156     2.089032   
21087   2019-11-30     0.940633     0.137439     0.424220     0.620865   
        2019-12-31     0.980076    -0.121022     0.263663     0.535294   
21214   2019-11-30     0.190229     0.022363     0.081954     0.122811   
        2019-12-31     0.140259    -0.090017    -0.009651     0.047674   

                            0.4          0.5          0.6          0.7  \
item_id timestamp                                                        
20001   2019-11-30  1517.190017  1606.161635  1691.608697  1780.838247   
        2019-12-31  1439.455506  1532.325155  1619.544369  1712.879930   
20002   2019-11-30  1359.883607  1439.086552  1515.881325  1597.188930   
        2019-12-31  1182.977076  1264.096869  1342.662521  1431.206024   
20003   2019-11-30   994.422002  1061.323981  1124.318012  1190.039940   
...                         ...          ...          ...          ...   
20995   2019-12-31     2.713818     3.319402     3.880894     4.505400   
21087   2019-11-30     0.787073     0.940633     1.086592     1.249452   
        2019-12-31     0.763726     0.980076     1.186562     1.417755   
21214   2019-11-30     0.157127     0.190229     0.220765     0.254224   
        2019-12-31     0.094967     0.140259     0.182843     0.229934   

                            0.8          0.9  
item_id timestamp                             
20001   2019-11-30  1888.273058  2042.488379  
        2019-12-31  1829.243851  1987.279963  
20002   2019-11-30  1697.683030  1844.922886  
        2019-12-31  1538.022268  1684.877489  
20003   2019-11-30  1266.213419  1386.105057  
...                         ...          ...  
20995   2019-12-31     5.320429     6.655886  
21087   2019-11-30     1.460386     1.808030  
        2019-12-31     1.711822     2.200117  
21214   2019-11-30     0.298116     0.370525  
        2019-12-31     0.291088     0.392014  

[1560 rows x 10 columns]

In [43]:
predictions_v1 = predictions.copy()
predictions_v1 = predictions_v1.reset_index()
predictions_v1 = predictions_v1[["item_id", "timestamp", "mean"]]
predictions_v1 = predictions_v1[predictions_v1.timestamp == "2019-12-31"]
predictions_v1 = predictions_v1.drop(columns = {"timestamp"})
predictions_v1 = predictions_v1.rename(columns = {"item_id":"product_id", "mean":"tn"})
predictions_v1.head(5)

,product_id,tn
1,20001,1542.149114
3,20002,1272.765049
5,20003,961.160696
7,20004,885.358688
9,20005,809.537310


In [44]:
predictions_v1.shape

(780, 2)

In [45]:
predictions_v1.to_csv("../entregable/outputs/prediccion_autogluon_sinFE_conSEMILLLERO.csv", sep = ",", index = False)